# Перевод из формата `.nc` в формат `.csv` данных о почве

Если качать данные из [ERA5-Land hourly data from 1950 to present](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview), то мы получим их в формате `.nc`. Для удобства переведем их в `.csv`, добавим `fips` коды.

[Код скачивания здесь](./download.py). Инструкция по настройке API на [сайте](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=overview).

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import h5py
import os
from pathlib import Path

PATH_ERA5 = Path("../../data/raw/ERA5-Land-Moisture")
PATH_SENTINEL = Path("../../data/raw/Sentinel/2019/")

Что делаем:
- переводим в dataframe для удобства;
- добавляем февраль в общую таблицу (он в отдельном файле, т.к. в нем <30 дней);
- сортируем по времени и координатам;
- добавляем FIPS коды (в снимках от Sentinel есть координаты округов);
- сохраняем в `.csv`.

In [ ]:
def transform_ds(
    *paths: Path, sort_by: list, fips: pd.DataFrame
) -> pd.DataFrame:
    """переводит из формата xarray в pd.dataframe. Добавляет fips коды. Некоторые снимки от Sentinel перекрывают друг друга.
    Здесь я поступил так: если один объект данных ERA5 относиться к разным
    снимкам Sentinel, то объект ERA5 дублируется.

    Args:
        paths (Path): пути к файлам .nc. В данном случае, для февраля и остальных месяцев два отдельных файла.
        sort_by (list): сортировка по столбцам
        fips_grid (pd.DataFrame): прямоугольные границы округов, вычисленных по данным Sentinel для соответствия данных.

    Returns:
        pd.DataFrame: полученный dataframe с fips кодами.
    """
    # data_feb_0.nc - данные отдельно для февраля, т.к. в нем 28 дней
    dss = [xr.open_dataset(path) for path in paths]

    df = pd.concat(
        [ds.to_dataframe().reset_index() for ds in dss], ignore_index=True
    )

    df.sort_values(by=sort_by, inplace=True)

    # Т.к. для таблицы влажности почвы `df_full` мы емеем точечные координаты,
    # то мы должны определить, к какому округу относится координата.
    # Для это мы и создали таблицу выше `fips`.
    df["fips"] = None
    df["state"] = None
    result_dfs = []
    # Проходим по каждой строке в fips и проверяем условия для df_full
    for idx, row in fips.iterrows():
        mask = (
            (df["longitude"] > row["left lon"])
            & (df["longitude"] < row["right lon"])
            & (df["latitude"] > row["lower lat"])
            & (df["latitude"] < row["upper lat"])
        )
        temp_df = df[mask].copy()
        temp_df["fips"] = row["fips"]
        temp_df["state"] = row["state"]
        result_dfs.append(temp_df)
    df_result = pd.concat(result_dfs)

    # Удаляем строки, для которых округи не определились
    df_result.dropna(axis=0, inplace=True)

    # Удаляем возможные дубликаты (если точка попадает ровно в один округ)
    df_result.drop_duplicates(inplace=True)
    return df_result

## Создаем таблицу координат и соответствующие им fips-коды

Таблица основана на данных [Sentinel](../../notebooks/1.1-data-review-sentinel.ipynb). Некоторые фотки перекрывают друг друга

Создадим нашу таблицу

In [3]:
cols = [
    "fips",
    "state",
    "left lon",
    "right lon",
    "lower lat",
    "upper lat",
]
features = []
# Проходимся по директориям штатов
for state_dir in PATH_SENTINEL.iterdir():
    # Достаточно по одному файлу из директории,
    # т.к. остальные - повторные того же штата
    with h5py.File(next(state_dir.iterdir()), "r") as h5file:
        for fips, item in h5file.items():
            first_date = next(iter(item))
            attrs = item[first_date]
            coordinates = attrs["coordinates"]
            # Находим общий bounding box для всех изображений
            # Левый нижний угол - минимальные lat и lon среди всех левых нижних углов
            lower_left_lat = np.min(coordinates[:, 0, 0])
            lower_left_lon = np.min(coordinates[:, 0, 1])

            # Правый верхний угол - максимальные lat и lon среди всех правых верхних углов
            upper_right_lat = np.max(coordinates[:, 1, 0])
            upper_right_lon = np.max(coordinates[:, 1, 1])

            # Добавляем найденные координаты
            features.append(
                [
                    fips,
                    attrs["state"][0].decode("utf-8"),
                    lower_left_lon,
                    upper_right_lon,
                    lower_left_lat,
                    upper_right_lat,
                ]
            )
fips = pd.DataFrame(data=features, columns=cols)
fips.shape
# должно быть 201. Ну почти - не суть

(200, 6)

In [4]:
fips.head(2)

,fips,state,left lon,right lon,lower lat,upper lat
0,19001,IOWA,-94.700629,-94.241593,41.157134,41.504148
1,19003,IOWA,-94.928459,-94.470603,40.899502,41.158508


In [5]:
swvl = transform_ds(
    PATH_ERA5 / "data_0.nc",
    PATH_ERA5 / "data_feb_0.nc",
    sort_by=["valid_time", "latitude", "longitude"],
    fips=fips,
)
swvl.head()

,valid_time,latitude,longitude,number,expver,src,swvl1,swvl2,swvl3,fips,state
2136,2018-01-01 11:00:00,41.171,-94.64,0,0001,9.802170e-08,0.280907,0.276657,0.280823,19001,IOWA
2137,2018-01-01 11:00:00,41.171,-94.54,0,0001,9.802170e-08,0.280746,0.275986,0.279739,19001,IOWA
2138,2018-01-01 11:00:00,41.171,-94.44,0,0001,8.940697e-08,0.280449,0.275566,0.278343,19001,IOWA
2139,2018-01-01 11:00:00,41.171,-94.34,0,0001,5.366746e-08,0.277565,0.272911,0.273750,19001,IOWA
2044,2018-01-01 11:00:00,41.271,-94.64,0,0001,8.940697e-08,0.279320,0.274940,0.279907,19001,IOWA


Разбиваем `valid_time` на `year`, `month`, `day`, `hour`

In [6]:
swvl["year"] = swvl["valid_time"].dt.year
swvl["month"] = swvl["valid_time"].dt.month
swvl["day"] = swvl["valid_time"].dt.day
swvl["hour"] = swvl["valid_time"].dt.hour

Удаляем лишние столбцы

In [7]:
swvl.drop(["number", "expver", "valid_time"], axis=1, inplace=True)

In [8]:
swvl.head(2)

,latitude,longitude,src,swvl1,swvl2,swvl3,fips,state,year,month,day,hour
2136,41.171,-94.64,9.802170e-08,0.280907,0.276657,0.280823,19001,IOWA,2018,1,1,11
2137,41.171,-94.54,9.802170e-08,0.280746,0.275986,0.279739,19001,IOWA,2018,1,1,11


In [9]:
swvl = swvl[
    [
        "year",
        "month",
        "day",
        "hour",
        "fips",
        "state",
        "latitude",
        "longitude",
        "src",
        "swvl1",
        "swvl2",
        "swvl3",
    ]
]

In [10]:
swvl.head(2)

,year,month,day,hour,fips,state,latitude,longitude,src,swvl1,swvl2,swvl3
2136,2018,1,1,11,19001,IOWA,41.171,-94.64,9.802170e-08,0.280907,0.276657,0.280823
2137,2018,1,1,11,19001,IOWA,41.171,-94.54,9.802170e-08,0.280746,0.275986,0.279739


## Сохранение

Структурируем по годам и штатам

In [11]:
states_info = pd.read_csv(
    "../../data/external/state_and_county_fips_master.csv"
)
states_info.head()

,fips,name,state
0,0,UNITED STATES,NaN
1,1000,ALABAMA,AL
2,1001,Autauga County,AL
3,1003,Baldwin County,AL
4,1005,Barbour County,AL


In [12]:
for year in swvl["year"].drop_duplicates():
    path_dir = PATH_ERA5 / str(year)
    os.mkdir(path_dir)
    for state in swvl["state"].drop_duplicates():
        state_fips, state_abbr = (
            states_info[states_info["name"] == state][["fips", "state"]]
            .iloc[0]
            .tolist()
        )
        state_fips = str(state_fips)[:2]
        target_dir = path_dir / state_abbr
        os.mkdir(target_dir)
        to_save = swvl[(swvl["year"] == year) & (swvl["state"] == state)]
        to_save.to_csv(
            target_dir / f"ERA5_{state_fips}_{state_abbr}_{year}.csv",
            index=False,
        )